In [420]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly_express as px
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [421]:
df = pd.read_csv("final_dataset.csv")

In [422]:
df.head(10)

,Unnamed: 0,Restaurant,Rating,reviewsCount,street,phone,categoryName,url,Latitude,Longitude,Restroom,Wifi Availabilty,Cost of 2 person,Payment Method
0,0,Haldiram's,4.2,12973,"Haldiram's - Restaurant & Retail Outlet, Plot ...",+91 712 252 2202,Fast food restaurant,https://www.google.com/maps/place/Haldiram's/@...,21.1367356,79.060505,Yes,Yes,905,ALL
1,1,Haldiram's,4.3,12347,"Unit No.60, Ground Floor, Anjuman Complex, Man...",+91 70300 00146,Family restaurant,https://www.google.com/maps/place/Haldiram's/@...,21.1635421,79.080170,Yes,Yes,562,ALL
2,2,Haldiram's Abhinandan,4.1,6920,"City Road, near City Post Office",+91 712 277 9187,Restaurant,https://www.google.com/maps/place/Haldiram's+A...,21.1526584,79.108589,Yes,Yes,700,ALL
3,3,Reddy's Gokul Brindavan,4.3,6279,"Coffee House Builiding, W High Ct Rd",+91 93731 02496,South Indian restaurant,https://www.google.com/maps/place/Reddy's+Goku...,21.1422,79.061052,Yes,Yes,852,ALL
4,4,The Chatori Street -Nagpur's 1st outdoor food ...,4.2,6137,"280, Central Bazar Road",+91 96736 14281,Food court,https://www.google.com/maps/place/The+Chatori+...,21.1332186,79.071868,Yes,Yes,712,ALL
5,5,Gayatri Bhojnalaya,4.3,6079,"W High Ct Rd, Behind Khandelwal Jewellers",+91 77091 09275,Marathi restaurant,https://www.google.com/maps/place/Gayatri+Bhoj...,21.1413035,79.061763,Yes,Yes,956,ALL
6,6,Checkers Restaurant VCA,4.2,5993,"23/24, VCA Ground, K Naidu Road, near Liberty ...",NaN,Restaurant,https://www.google.com/maps/place/Checkers+Res...,21.1578596,79.076661,Yes,Yes,559,ALL
7,7,Uttar Dakshin by Naivedhyam,4.3,5942,Mount Rd Ext,+91 93268 81014,South Indian restaurant,https://www.google.com/maps/place/Uttar+Dakshi...,21.1588564,79.079894,Yes,No,949,ALL
8,8,Vishnuji Ki Rasoi,4.1,5807,Central Bazar Road,+91 93703 27557,Marathi restaurant,https://www.google.com/maps/place/Vishnuji+Ki+...,21.1313664,79.065387,Yes,Yes,519,ALL
9,9,Barbeque Nation - Nagpur - Eternity Mall,4.4,5223,"2nd Floor Eternity Mall, Amravati Rd",+91 80 6902 8737,Barbecue restaurant,https://www.google.com/maps/place/Barbeque+Nat...,21.1429634,79.080415,Yes,Yes,680,ALL


In [423]:
df.shape

(1996, 14)

In [424]:
df.index

RangeIndex(start=0, stop=1996, step=1)

In [425]:
df.columns

Index(['Unnamed: 0', 'Restaurant', 'Rating', 'reviewsCount', 'street', 'phone',
       'categoryName', 'url', 'Latitude', 'Longitude', 'Restroom',
       'Wifi Availabilty', 'Cost of 2 person', 'Payment Method'],
      dtype='object')

In [426]:
df=df.drop(['Unnamed: 0','Unnamed: 0'],axis=1)

In [427]:
#df = df.drop_duplicates(subset=['title'])

In [428]:
df.shape

(1996, 13)

Arranged the table on the basis of reviewcount and Rating

In [429]:
import numpy as np
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
mean_rating = df['Rating'].mean()
df['Rating'].fillna(mean_rating, inplace=True)
df['Rating'] = df['Rating'].astype(float)

In [430]:
print(df['reviewsCount'].dtype)

int64


In [431]:
df = df.sort_values(by=['reviewsCount','Rating'],ascending=False)[:] 

In [432]:
df = df.drop_duplicates(subset=['url'])

In [433]:
df = df.reset_index(drop=True)

In [434]:
df.head(100)

,Restaurant,Rating,reviewsCount,street,phone,categoryName,url,Latitude,Longitude,Restroom,Wifi Availabilty,Cost of 2 person,Payment Method
0,Haldiram's,4.2,12973,"Haldiram's - Restaurant & Retail Outlet, Plot ...",+91 712 252 2202,Fast food restaurant,https://www.google.com/maps/place/Haldiram's/@...,21.1367356,79.060505,Yes,Yes,905,ALL
1,Haldiram's,4.3,12347,"Unit No.60, Ground Floor, Anjuman Complex, Man...",+91 70300 00146,Family restaurant,https://www.google.com/maps/place/Haldiram's/@...,21.1635421,79.080170,Yes,Yes,562,ALL
2,Haldiram's Abhinandan,4.1,6920,"City Road, near City Post Office",+91 712 277 9187,Restaurant,https://www.google.com/maps/place/Haldiram's+A...,21.1526584,79.108589,Yes,Yes,700,ALL
3,Reddy's Gokul Brindavan,4.3,6279,"Coffee House Builiding, W High Ct Rd",+91 93731 02496,South Indian restaurant,https://www.google.com/maps/place/Reddy's+Goku...,21.1422,79.061052,Yes,Yes,852,ALL
4,The Chatori Street -Nagpur's 1st outdoor food ...,4.2,6137,"280, Central Bazar Road",+91 96736 14281,Food court,https://www.google.com/maps/place/The+Chatori+...,21.1332186,79.071868,Yes,Yes,712,ALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Gurpal Foods,3.9,1150,"Plot No 14, Kamptee Road, Opp Tirpude Hospital",+91 93725 96161,Punjabi restaurant,https://www.google.com/maps/place/Gurpal+Foods...,21.1724472,79.100048,No,Yes,697,CASH AND UPI
96,Villa 167 Cafe,4.3,1146,"Plot No 167, Abhyankar Nagar Rd",+91 78208 83985,Cafe,https://www.google.com/maps/place/Villa+167+Ca...,21.1226156,79.057234,Yes,Yes,987,ALL
97,Villa 167 Cafe,4.3,1146,"plot no 167 , Villa, 167, Abhyankar Nagar Rd",+91 78208 83985,Cafe,https://www.google.com/maps/place/Villa+167+Ca...,21.1267214,79.057502,Yes,Yes,945,ALL
98,Cafe Coffee Day,4.0,1141,"Shop no 9,10,11, VCA Stadium Complex, Rabindra...",+91 1800 102 5093,Cafe,https://www.google.com/maps/place/Cafe+Coffee+...,21.1570552,79.077172,Yes,Yes,906,ALL


In [435]:
df.shape

(1996, 13)

In [436]:
df.columns

Index(['Restaurant', 'Rating', 'reviewsCount', 'street', 'phone',
       'categoryName', 'url', 'Latitude', 'Longitude', 'Restroom',
       'Wifi Availabilty', 'Cost of 2 person', 'Payment Method'],
      dtype='object')

In [437]:
top_restaurant = df.sort_values(['reviewsCount','Rating'],ascending=False) [:20]

In [438]:
df.columns

Index(['Restaurant', 'Rating', 'reviewsCount', 'street', 'phone',
       'categoryName', 'url', 'Latitude', 'Longitude', 'Restroom',
       'Wifi Availabilty', 'Cost of 2 person', 'Payment Method'],
      dtype='object')

In [439]:
df = df[df['Latitude'] != '#VALUE!']
df = df[df['Longitude'] != '#VALUE!']
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# K-Mean Clusttering

In [440]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

coords = df[['Latitude','Longitude']]
distortions = []
K = range(1, 25)

for k in K:
    kmeans_model = KMeans(n_clusters=k)
    kmeans_model = kmeans_model.fit(coords)
    distortions.append(kmeans_model.inertia_)

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the w

# Silhouette Method

In [441]:
from sklearn.metrics import silhouette_score

sil = []
kmax = 50

# dissimilarity would not be defined for a single cluster, thus, minimum number of clu

for k in range(2, kmax+1):
    kmeans = KMeans (n_clusters = k).fit(coords)
    labels = kmeans.labels_
    sil.append(silhouette_score(coords, labels, metric = 'euclidean'))

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the w

In [442]:
sil

[0.3814804557216189,
 0.4128995124174254,
 0.3882889108799941,
 0.3528798376571443,
 0.39210695371596826,
 0.39692055275322513,
 0.4029089611703391,
 0.41094756774945174,
 0.4106900140105129,
 0.41050548525797725,
 0.41782236413779594,
 0.42100686648935437,
 0.43282546859222726,
 0.4300698133926965,
 0.4278852693412075,
 0.4292324522799866,
 0.41946562815727506,
 0.429836994877078,
 0.43115650644987896,
 0.4329341552293131,
 0.4285225963112591,
 0.4370124833354928,
 0.43820816071703383,
 0.4308905335544436,
 0.4460184189791285,
 0.4435877427588015,
 0.4371671633047711,
 0.44148195892255115,
 0.44827635457146464,
 0.4340471891559203,
 0.44448598135890377,
 0.44797662855700143,
 0.4442649454112828,
 0.4451445432726267,
 0.44488452489850056,
 0.44803787678052326,
 0.4384211693585574,
 0.4438688576013093,
 0.4482080143916297,
 0.42793107738977954,
 0.430291268770884,
 0.44213685727195756,
 0.44368722953647544,
 0.4444282834613857,
 0.4341007287738665,
 0.44025797215151136,
 0.4457590710911

In [443]:
kmeans = KMeans (n_clusters=7, init='k-means++')
kmeans.fit(coords)
y = kmeans.labels_
print("k = 7", "silhouette_score", silhouette_score(coords, y, metric='euclidean'))

k = 7 silhouette_score 0.3968347391775048


c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [444]:
df['cluster'] = kmeans.predict(df[[ 'Latitude','Longitude']]) 

In [445]:
px.scatter_mapbox(df,lat="Latitude", lon="Longitude", color="cluster", size='reviewsCount' ,
                  hover_data=['Restaurant','Latitude','Longitude'], zoom=10, width=1200, height=800)

# Location-Based Recommendation

In [446]:
top_restaurnant_nagpur = df.sort_values(by=['reviewsCount','Rating'], ascending=False)


In [447]:
df.columns

Index(['Restaurant', 'Rating', 'reviewsCount', 'street', 'phone',
       'categoryName', 'url', 'Latitude', 'Longitude', 'Restroom',
       'Wifi Availabilty', 'Cost of 2 person', 'Payment Method', 'cluster'],
      dtype='object')

In [448]:

# def recommend_restaurants(latitude, longitude):
#     user_location = np.array([latitude,longitude]).reshape(1, -1)
#     cluster = kmeans.predict(user_location)[0]
#     cluster_restaurants = df[df['cluster'] == cluster]
#     return df[df["cluster"]==cluster].iloc[0:100][['Restaurant', 'Latitude', 'Longitude','Rating','categoryName','url']];
   

In [449]:
# import math

# def calculate_distance(lat1, lon1, lat2, lon2):
#     # Convert latitude and longitude from degrees to radians
#     lat1_rad = math.radians(lat1)
#     lon1_rad = math.radians(lon1)
#     lat2_rad = math.radians(lat2)
#     lon2_rad = math.radians(lon2)


#     # Difference between the latitudes and longitudes
#     d_lat = lat2_rad - lat1_rad
#     d_lon = lon2_rad - lon1_rad


#     # Haversine formula
#     a = math.sin(d_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(d_lon/2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

#     # Radius of the Earth (in kilometers)
#     radius = 6371

#     # Calculate the distance
#     distance = radius * c
#     return distance


In [450]:
category = [
    'Fast food restaurant',
    'Family restaurant',
    'South Indian restaurant',
    'Food court',
    'Marathi restaurant',
    'Barbecue restaurant',
    'Bakery',
    'North Indian restaurant',
    'Indian restaurant',
    'Bar & grill',
    'Mughlai restaurant',
    'Non vegetarian restaurant',
    'Pizza restaurant',
    'Vegetarian restaurant',
    'Rajasthani restaurant',
    'Breakfast restaurant',
    'Italian restaurant',
    'Punjabi restaurant',
    'Candy store',
    'Chicken restaurant',
    'Fine dining restaurant',
    'Biryani restaurant',
    'Sandwich shop',
    'Middle Eastern restaurant',
    'Takeout restaurant',
    'Small plates restaurant',
    'American restaurant',
    'Chinese restaurant',
    'Dhaba',
    'Juice shop',
    'Grill',
    'Shawarma restaurant',
    'Bar',
    'Momo restaurant',
    'Asian restaurant',
    'Buffet restaurant',
    'Bengali restaurant',
    'Country food restaurant',
    'Hyderabadi restaurant',
    'Seafood restaurant',
    'Dessert shop',
    'Health food restaurant',
    'Vegan restaurant',
    'Ice cream shop',
    'Hamburger restaurant',
    'Diner',
    'Seafood donburi restaurant',
    'Gujarati restaurant',
    'Snack bar',
    'Indian takeaway',
    'Fish & chips restaurant',
    'Hoagie restaurant',
    'Cold noodle restaurant',
    'Cape Verdean restaurant',
    'Afghan restaurant',
    'Tiffin center',
    'Continental restaurant',
    'Mandarin restaurant',
    'Chinese noodle restaurant',
    'Fondue restaurant',
    'Sundae restaurant',
    'Fish restaurant',
    'Rice restaurant',
    'Korean restaurant'
]



In [451]:
import math
def recommend(latitude,longitude,rad,user_choice,restroom,wifi):

    user_location = np.array([latitude,longitude]).reshape(1, -1)
    cluster = kmeans.predict(user_location)[0]
    recommended_restaurants = df[df["cluster"]==cluster].iloc[0:200][['Restaurant', 'Latitude', 'Longitude','Rating','categoryName','url','Restroom','Wifi Availabilty','Cost of 2 person','Payment Method']]
   

    recommended_restaurants['Distance'] = None

    fixed_lat = latitude
    fixed_lon = longitude
    radius = rad
    # List of other coordinates
    for index, row in recommended_restaurants.iterrows():
        restaurant_latitude = row['Latitude']
        restaurant_longitude = row['Longitude']

        lat1_rad = math.radians(fixed_lat)
        lon1_rad = math.radians(fixed_lon)
        lat2_rad = math.radians(restaurant_latitude)
        lon2_rad = math.radians(restaurant_longitude)


    # Difference between the latitudes and longitudes
        d_lat = lat2_rad - lat1_rad
        d_lon = lon2_rad - lon1_rad


    # Haversine formula
        a = math.sin(d_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(d_lon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # Radius of the Earth (in kilometers)
        radius = 6371

    # Calculate the distance
        distance = radius * c
        recommended_restaurants.at[index,'Distance']=distance 



    
    filtered_dist = recommended_restaurants[(recommended_restaurants['Distance'] <= rad)]

    print(filtered_dist)

    if restroom == 1 and wifi == 1 :
        filtered_dist = filtered_dist[(filtered_dist['Restroom'] == 'Yes') &
                                    (filtered_dist['Wifi Availabilty'] == 'Yes')]
    elif restroom == 1 and wifi == 0 :
        filtered_dist = filtered_dist[(filtered_dist['Restroom'] == 'Yes')]
    elif restroom == 0 and wifi == 1 :
        filtered_dist = filtered_dist[(filtered_dist['Wifi Availabilty'] == 'Yes')]
 
    
 
    print(filtered_dist)


    
    bychoice = filtered_dist[filtered_dist['categoryName'].str.contains(user_choice, case=False)]

    bychoice = pd.concat([bychoice, filtered_dist.head(10)])



    

    print("Top 10 Restaurant Near you")
    print(bychoice[['Restaurant', 'categoryName','Rating','Distance','url','Restroom','Wifi Availabilty','Payment Method']].head(10))

    bychoice = bychoice.reset_index(drop=True)

    return bychoice[['Restaurant', 'categoryName','Rating','Distance','url','Latitude','Longitude','Restroom','Wifi Availabilty','Payment Method']].head(10).to_json(orient='columns')





In [452]:

import cloudpickle
cloudpickle.dump(recommend,open('model.pkl','wb'))

In [453]:
recommend(21.1750198,79.1203504,3,'Cafe',1,1)


                                             Restaurant   Latitude  Longitude   
88                                     Hotel Al Rahman.  21.178522  79.131802  \
208   Jaiswal Restaurant/Haldiram Sweets,Snacks,Brea...  21.179375  79.117884   
298                                    Zaika Restaurant  21.183517  79.115713   
306                              New taj biryani centre  21.174380  79.110735   
317                Afghan Darbar Restaurant & Takeaways  21.182665  79.124052   
...                                                 ...        ...        ...   
1717                                        Nasta point  21.189997  79.126312   
1729                            ChongQing Cafe & Restro  21.162718  79.113376   
1737                                  Hide & Seek Caffe  21.183852  79.132737   
1739                                       Yogesh Hotel  21.197014  79.117982   
1741                                    Chotu Pani Puri  21.174687  79.138543   

        Rating          cat

c:\Users\RITESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning:

X does not have valid feature names, but KMeans was fitted with feature names



'{"Restaurant":{"0":"Aroma Cafe","1":"Delicioso cafe","2":"Triple 2 cafe","3":"The Cafe - Taste Dil Se","4":"Nilesh Service","5":"Chandravanshi Juice","6":"Harsh Chai & Nashta Cente","7":"Sapphire\'s Cafe","8":"Hotel Al Rahman.","9":"Zaika Restaurant"},"categoryName":{"0":"Cafe","1":"Cafe","2":"Cafe","3":"Cafe","4":"Cafe","5":"Cafe","6":"Cafe","7":"Cafe","8":"Mughlai restaurant","9":"Restaurant"},"Rating":{"0":4.3,"1":4.9,"2":5.0,"3":5.0,"4":4.191498741,"5":4.191498741,"6":4.191498741,"7":4.191498741,"8":4.2,"9":3.9},"Distance":{"0":1.5509019776,"1":0.5510665334,"2":1.4885336716,"3":1.7227313612,"4":0.44062159,"5":2.9464617408,"6":2.1859334305,"7":0.5176254452,"8":1.24965724,"9":1.0601730535},"url":{"0":"https:\\/\\/www.google.com\\/maps\\/place\\/Aroma+Cafe\\/@21.1626597,79.1134203,17z\\/data=!3m1!4b1!4m6!3m5!1s0x3bd4c7bc234408cd:0xd6c4fbacd175968c!8m2!3d21.1626597!4d79.1134203!16s%2Fg%2F11k2b4fyvj?hl=en","1":"https:\\/\\/www.google.com\\/maps\\/place\\/Delicioso+cafe\\/@21.1716088,79

In [454]:
# recommend(21.1730198,79.1003504,20,'Fast Food')

In [455]:
# recommend(21.120712,78.9900825,10,'Vegetarian')


In [456]:
# recommend(21.120712,78.9900825,6,'Biryani')

In [457]:
# final = recommend(21.17121463402801, 79.12374868158489,10,'North Indian')
# px.scatter_mapbox(final, lat="Latitude", lon="Longitude", color="Rating",
                #   size_max=30, zoom=3, width=1200, height=800)

In [458]:
# final = recommend(20.7533238,78.5911708,100,'Fast Food')

In [459]:
# user_latitude = float(input("Enter your latitude: "))
# user_longitude = float(input("Enter your longitude: "))
# distance = float(input("Enter Distance radius: "))
# choice = input("Enter your choice: ")

# final = recommend(user_latitude,user_longitude,distance,choice)

# px.scatter_mapbox(final, lat="Latitude", lon="Longitude", color="Rating",
#                   size_max=30, zoom=3, width=1200, height=800)
# final